In [1]:
!pip install -q pyyaml h5py

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.6.0


In [75]:
(tr_img, tr_cls), (te_img, te_cls) = tf.keras.datasets.mnist.load_data()

In [4]:
tr_img.shape, te_img.shape

((60000, 28, 28), (10000, 28, 28))

In [76]:
tr_cls = tr_cls[:1000]
te_cls = te_cls[:1000]

In [77]:
tr_img = tr_img[:1000].reshape(-1, 28 * 28) / 255.0
te_img = te_img[:1000].reshape(-1, 28 * 28) / 255.0

In [78]:
def create_model() :
  model = tf.keras.models.Sequential([
          keras.layers.Dense(512, activation = 'relu', input_shape = (784, )),
          keras.layers.Dropout(.2),
          keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

  return model

In [92]:
model = create_model()

In [93]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### Model Checkpoint 활용

- 모델을 학습 중간 혹은 종료시 저장한다.
- 저장된 모델을 다시 불러와 추가적인 학습이 가능하다.

In [94]:
checking_path = 'training_1/'
checking_dir = os.path.dirname(checking_path)

In [95]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath = checking_dir, save_weights_only = True, verbose = 1)

In [96]:
model.fit(tr_img, tr_cls, epochs = 10, validation_data = (te_img, te_cls), callbacks = [mc])

Epoch 1/10
32/32 [==============================] - 1s 10ms/step - loss: 4.2822 - acc: 0.2850 - val_loss: 2.0749 - val_acc: 0.2810

Epoch 00001: saving model to training_1
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 2.2040 - acc: 0.2680 - val_loss: 2.1092 - val_acc: 0.2690

Epoch 00002: saving model to training_1
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 2.2608 - acc: 0.2540 - val_loss: 2.2013 - val_acc: 0.2830

Epoch 00003: saving model to training_1
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 2.2565 - acc: 0.2460 - val_loss: 2.2059 - val_acc: 0.2830

Epoch 00004: saving model to training_1
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 2.2542 - acc: 0.2440 - val_loss: 2.2059 - val_acc: 0.2830

Epoch 00005: saving model to training_1
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 2.2427 - acc: 0.2410 - val_loss: 2.2015 - val_acc: 0.2830

Epoch 00006: 

In [97]:
model = create_model()

In [98]:
loss, acc = model.evaluate(te_img, te_cls, verbose = 2)

32/32 - 0s - loss: 11.7028 - acc: 0.0480


#### model.load_weights(checking_path) 하고 동일하게 evaluate하면 저장된 가중치 사용해서 성능이 더 좋다.

In [ ]:
mc = tf.keras.callbacks.ModelCheckpoint(
    file_path = checking_path,
    verbose = 1,
    save_weights_only = True,
    period = 5
)

### 다섯 번의 에포크마다 체크포인트를 저장하라는 의미다.

#### 수동으로 가중치 저장하기

- model.save_weights

In [101]:
model.save_weights('mc')

model.load_weights('mc')

In [102]:
loss, acc = model.evaluate(te_img, te_cls, verbose = 2)

32/32 - 0s - loss: 11.7028 - acc: 0.0480


#### model.save는 전체 모델을 저장한다.

- 모델의 구조
- 가중치
- 훈련설정

SaveModel or HDF5로 저장된다.

In [103]:
model = create_model()

In [104]:
model.fit(tr_img, tr_cls, epochs = 5)

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 3.1584 - acc: 0.3170
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2450 - acc: 0.3200
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2519 - acc: 0.3340
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2496 - acc: 0.3280
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2496 - acc: 0.3280


In [105]:
model.save('save_model')

INFO:tensorflow:Assets written to: save_model/assets


In [115]:
new_model = tf.keras.models.load_model('save_model')

In [116]:
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [117]:
loss, acc = new_model.evaluate(te_img, te_cls, verbose = 2)

32/32 - 0s - loss: 2.2404 - acc: 0.2800


#### HDF5

In [109]:
model = create_model()

In [110]:
model.fit(tr_img, tr_cls, epochs = 5)

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 4.4001 - acc: 0.4570
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 2.1998 - acc: 0.3370
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2206 - acc: 0.2520
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2451 - acc: 0.2640
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 2.2312 - acc: 0.2680


In [111]:
model.save('my_model.h5')

In [112]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [114]:
loss, acc = new_model.evaluate(te_img, te_cls, verbose = 2)

32/32 - 0s - loss: 2.2611 - acc: 0.2420
